<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/Transfer_Learning_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch 
  print(f"You are on CoLaB with torch version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t: float = 123.461)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>time formating: please wait...\n>>>time elapse is: {time_fmt()}")


Mounted at /content/drive
You are on CoLaB with torch version: 1.8.1+cu101
>>>time formating: please wait...
>>>time elapse is: 0: 02: 03.00


In [2]:
#In this notebook we are going to impliment a transfer learning procidure on cifar10 data in pytorch

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from tqdm import tqdm
import time, datetime, sys
import torchvision

In [20]:
#Load the VGGNET from torch
model = torchvision.models.vgg16(pretrained = True)

In [21]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [22]:
#We modify the modelby removing the average pooling layer since cifar10 images will shrink to 1x1
#we also modify the classifier to be 10 classes 

In [23]:
#Inplace of average pooling layer we want the model to return output of the same shape as input
class ID(nn.Module):
  def __init__(self):
    super(ID, self).__init__()
  def forward(self,x):
    return x
    


In [24]:
#Apply this class to our model class we have:
model.avgpool = ID()

In [25]:
#Define new classifier
class CLF(nn.Module):
  def __init__(self):
    super(CLF, self).__init__()
    self.fc1 = nn.Linear(in_features = 512, out_features = 256)
    self.fc2 = nn.Linear(in_features = 256, out_features = 128)
    self.out = nn.Linear(in_features = 128, out_features = 10)
  def forward(self, x):
    x = nn.functional.relu(self.fc1(x))
    x = nn.functional.relu(self.fc2(x))
    return self.out(x)

In [26]:
#Apply the classifier
model.classifier = CLF()

In [27]:
model.to(device = device) #save model to device

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [28]:
#Hyperparameters:
batch_size = 64
EPOCHS = 10
learning_rate = 1e-3


In [29]:
#Loading and split the data into batches of size 64
train_data = datasets.CIFAR10(root = '/train_data', train = True, transform = transforms.ToTensor(), download = True)
test_data = datasets.CIFAR10(root = '/test_data', train = False, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_data, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
x_batch, y_batch = next(iter(train_loader))
print(f"x_batch_shape: {x_batch.shape}: y_batch_shape: {y_batch.shape}")

Files already downloaded and verified
Files already downloaded and verified
x_batch_shape: torch.Size([64, 3, 32, 32]): y_batch_shape: torch.Size([64])


In [30]:
#Get the loss and optimizer objects:
loss_obj = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)

In [31]:
#Training loop:
tic = time.time()
for epoch in range(EPOCHS):
  print(f"\n>>>Training starts for epoch: {epoch + 1}\n>>>Please wait while the model is training\n>>>Maintain Screen Activity.......")
  for idx, (data, target) in enumerate(tqdm(train_loader)):
    #assign data to GPU
    data = data.to(device = device)
    target = target.to(device = device)
    #forward pass here:
    preds = model(data)
    train_loss = loss_obj(preds, target)
    #backward pass
    optimizer.zero_grad()
    train_loss.backward()
    #gradient descent with adam optimizer
    optimizer.step()

#Tracking performance on training and validation data
def model_performance(loader, model):
  if loader.dataset.train:
    print(f"\n>>>Checking the performance over the training data\n>>>please wait and maintain screen activity...")
  else:
    print(f"\n>>>Checking the performance over the validation data\n>>>please wait and maintain screen activity...")
  num_examples = 0
  num_correct = 0
  model.eval()
  #no need to train the model again
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      preds = model(x)
      _, predictions = preds.max(1)#fetch the maximum prob among the class of 10 probability values
      num_correct+=(predictions == y).sum()#total correctly predicted values
      num_examples+=predictions.size(0)#fetch all examples (batch_size = 64 in this case)
  model.train()
  return float(num_correct/num_examples)*100 #compute the accuracy in percentages
toc = time.time()
print(f"\n>>>The accuracy over the training data: {model_performance(train_loader, model):.2f}")
print(f"\n>>>The accuracy over the validation data: {model_performance(test_loader, model):.2f}")
print(f"\n>>>Total time for training and validation: {time_fmt(toc - tic)}")

  1%|          | 5/782 [00:00<00:18, 42.43it/s]


>>>Training starts for epoch: 1
>>>Please wait while the model is training
>>>Maintain Screen Activity.......


  1%|          | 5/782 [00:00<00:17, 45.38it/s]


>>>Training starts for epoch: 2
>>>Please wait while the model is training
>>>Maintain Screen Activity.......


  1%|          | 5/782 [00:00<00:18, 42.26it/s]

...


>>>Training starts for epoch: 9
>>>Please wait while the model is training
>>>Maintain Screen Activity.......


  1%|          | 5/782 [00:00<00:17, 45.35it/s]


>>>Training starts for epoch: 10
>>>Please wait while the model is training
>>>Maintain Screen Activity.......


100%|██████████| 782/782 [00:18<00:00, 42.91it/s]



>>>Checking the performance over the training data
>>>please wait and maintain screen activity...

>>>The accuracy over the training data: 94.29

>>>Checking the performance over the validation data
>>>please wait and maintain screen activity...

>>>The accuracy over the validation data: 83.14

>>>Total time for training and validation: 0: 03: 02.00
